<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [9]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv
from datetime import datetime
import re

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 17 #needs to be addapted to 1 nr in the series
sample_nr_high = 19 #needs to be addapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [10]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)

        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)

            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

14.71	14.65	14.77	3966684010.158	100.0	1825824216.047	100.0
17.49	17.4	17.56	5104201584.62	16.67	2426938112.0	18.53
20.8	20.74	20.88	34327567007.934	100.0	13935504069.075	100.0
23.93	23.86	24.01	9773080566.616	31.91	4658581504.0	35.58
29.64	29.58	29.78	3777665019.769	12.34	1561089293.712	11.92
32.17	32.12	32.4	5812593872.55	18.98	2277990100.526	17.4
34.56	34.5	34.84	6156155001.679	20.1	2169597629.686	16.57
14.7	14.65	14.76	2239831054.446	100.0	1049964595.419	100.0
17.49	17.41	17.56	2390000805.998	23.13	1178118256.656	26.56
20.82	20.75	20.87	39123238211.884	100.0	16067162255.855	100.0
23.93	23.86	24.04	3735343330.127	36.15	1729183392.561	38.99
29.64	29.59	29.9	1277439860.587	12.36	492662889.096	11.11
32.18	32.11	32.48	1478428543.323	14.31	553023625.617	12.47
34.57	34.5	34.91	1451251157.288	14.05	482402964.952	10.88


The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [11]:
# Loop over all the designated sample files. Currently works but i am not shure if it might be just exactly the same as the first one, it works but not iterative...
for n in range(sample_nr_low, sample_nr_high):
    divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
    directory = '/content/drive/My Drive/quantitation_output'

    if not os.path.isfile(divisor_file):
        print(f"Divisor file does not exist: {divisor_file}")
        continue  # Skip to the next iteration if the divisor file is missing

    divisor_row = pd.read_csv(divisor_file, sep=',', header=None)
    csv_files = glob.glob(os.path.join(directory, f'*_row*.csv'))

    # Process each CSV file
    for csv_file in csv_files:
        if csv_file == divisor_file or '_ratio' in csv_file:
            continue  # Skip the divisor file itself and ratio files
        data_df = pd.read_csv(csv_file, sep=',', header=None)
        data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric

        # Perform division on the data
        divided_data_df = data_df.div(divisor_row, axis='columns')

        # Create a new file name with the "_ratio" tag
        new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

        divided_data_df.to_csv(new_file_name, sep=',', header=False, index=False)
        control_import_df = pd.read_csv(new_file_name, sep=',', header=None)

        # Compare the recalculated values with the content of the ratio file
        if control_import_df.equals(divided_data_df):
            print(f"The files match: {new_file_name}")
        else:
            print(f"The files do not match: {new_file_name}")

        print(control_import_df)
        print(divided_data_df)
        print('****************************************************')

The files do not match: /content/drive/My Drive/quantitation_output/Sample_17_row1_ratio.csv
          0         1         2         3    4        5    6
0  0.707212  0.706365  0.707375  0.115554  1.0  0.13102  1.0
          0         1         2         3    4        5    6
0  0.707212  0.706365  0.707375  0.115554  1.0  0.13102  1.0
****************************************************
The files do not match: /content/drive/My Drive/quantitation_output/Sample_17_row2_ratio.csv
          0         1         2         3       4         5       6
0  0.840865  0.838959  0.840996  0.148691  0.1667  0.174155  0.1853
          0         1         2         3       4         5       6
0  0.840865  0.838959  0.840996  0.148691  0.1667  0.174155  0.1853
****************************************************
The files do not match: /content/drive/My Drive/quantitation_output/Sample_17_row4_ratio.csv
          0         1         2         3       4         5       6
0  1.150481  1.150434  1.149904

The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information.

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [12]:

current_date = datetime.today().strftime('%Y-%m-%d')
output_dir = '/content/drive/My Drive/quantitation_export/'
#header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]
header_row2 = ['Analyte', 'Intercept', 'Slope', 'Value', 'Concentration [microg/ml]']

row_descriptors = ['place holder','Dimethylphthalate', 'Diethylphthalate', 'Internal standard', 'Dibutylphthalate', 'Benzylbutylphthalate', 'Bis(2-ethyl-hexyl)phthalate', 'Di-n-octylphthalate']
file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output.csv'
    header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i in range(1, 8):
          if i == 3:
              continue
          print(f'Sample nr: {n}')
          print(f'Compound iteration: {i}')
          linreg_df = pd.read_csv(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
          print(f'linear function information: {linreg_df}')
          descriptor = row_descriptors[i]
          print(f'Descriptor: {descriptor}')
          file_path = file_pattern.format(n, i)
          if not os.path.isfile(file_path):
            print(f"File not found for Sample {n}, Row {i}")
            continue

          sample_df = pd.read_csv(file_pattern.format(n, i), header=None, delimiter=',')  # Corrected line
          print(f'Sample dataframe: {sample_df}')
          intercept_str = linreg_df['Intercept'].values[0]
          slope_str = linreg_df['Slope'].values[0]
          intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
          slope = float(re.findall(r'[-\d.]+', slope_str)[0])
          fourth_column = sample_df.iloc[:, 3]  # Accessing the 4th column
          value = float(fourth_column.iloc[0])  # Assuming you want the value from the first row (index 0)
          concentration = slope * value + intercept
          print(f"Calculated concentration: {concentration}")
          row_data = [descriptor, intercept, slope, value, concentration]
          writer.writerow(row_data)

          # Additional print statements for debugging
          print(f"Sample DataFrame shape: {sample_df.shape}")
          print(f"Linreg DataFrame shape: {linreg_df.shape if linreg_df is not None else None}")
          print("------------------------------------------------------------------------")


Sample nr: 17
Compound iteration: 1
linear function information:        Intercept            Slope  R2 Value
0  [-0.01726225]  [[16.48584551]]  0.967163
Descriptor: Dimethylphthalate
Sample dataframe:           0         1         2         3    4         5    6
0  0.706532  0.706024  0.707714  0.101389  1.0  0.113637  1.0
Calculated concentration: 1.6542287299717964
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 17
Compound iteration: 2
linear function information:        Intercept            Slope  R2 Value
0  [-0.00909398]  [[13.87985743]]  0.978623
Descriptor: Diethylphthalate
Sample dataframe:           0         1         2         3       4        5       6
0  0.840058  0.838554  0.841399  0.130465  0.1667  0.15105  0.1853
Calculated concentration: 1.8017374727754443
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------